# 4. Algorithmic question 

A number ***n*** of kids are in a camp. Between some ***k*** pairs of them (a kid can be part of more than one pairs) there are often fights. At night there are two dormitories where the kids can sleep. We want, if possible, to assign each kid in one of the two dormitories in such a way that each pair of kids that fights often is assigned to a different dormitory. (There are no space problems and the two dormitories can have different number of kids.)

Give an algorithm that is linear in ***n*** and ***k*** that is able to answer whether such an assignment is possible and, if so, return one.

We formalize the given instruction using a graph in which every node is a kid, and every edge is an "hate relationship" between them, i.e. two kids are connected only if they fight often. So we have a graph $(V,E)$ with $n$ nodes and $k$ edges, the problem then translates in being able to split $V$ into two subset with the following property: if two nodes are connected by an edge, they are not in the same subset.

We'll give a description of the algorithm in pseudocode and then we'll implement it in python. To start, we'll suppose that the input graph is connected, if it isn't it's sufficient to apply the algorithm to every connected component and then join the eventual solution (which is trivial, since two nodes coming from disconnected component are, by definition, not connected).

Our algorithm is based on the depth-first search algorithm, so let ```G``` be a connected graph.

```
kidOrganizer(G):
    Let Q be an empty stack
    
    dorm[v] = NotAssigned   for every node v    # array that says to which dorm the kid v should be assigned (if dorm 0 or dorm 1)
    
    s = random starting node in G
    dorm[s] = 0    # we assign the first kid to dorm 0
    Q.push(s)
    
    while Q is not empty:
        
        v = Q.pop()
        
        for u in neighbours(v):
            if dorm[u] is NotAssigned:
                dorm[u] = (dorm[v] + 1)%2  # changes the dorm from 0 to 1, or from 1 to 0
                Q.push(u)
                
            elif dorm[u] == dorm[v]:
                print('There is no solution for the given graph, the kids are doomed to fight until just one remains. Welcome to the thunderdome.')
                return(None)
    
    return(dorm)
```

For a connected graph $G$ with $n$ nodes and $k$ edges, this algorithm has a $O(k)$ complexity.

We will now give a python implementation of the algorithm in the general case of a disconnected graph. We will assume that the input graph $G$ is passed as an adjacency list.

In [23]:
def general_kidOrganizer(G):
    '''
    INPUT: G is the adjacency list of the input graph G
    
    OUTPUT: list that contains in the i-th position the dorm (0 or 1) to which the i-th kid should be assigned
    '''
    
    n = len(G)
    
    dorm = [-1]*n   # list that says to which dorm the kid v should be assigned (if dorm 0 or dorm 1)
                    # -1 is 'not assigned'
    
    total_nodes = 0
    while (total_nodes<n):
        
        s = getNextNode(dorm)
        
        processed_nodes = kidOrganizer(G, dorm, s)
        
        if processed_nodes == 0:
            print('There is no solution for the given graph, the kids are doomed to fight until just one (or maybe more) remains.\n\nWelcome to the thunderdome.\n')
            return([])
        
        total_nodes += processed_nodes
    
    return(dorm)

def getNextNode(dorm):
    '''
    retrieves the first non assigned node in the graph
    '''
    
    for idx, dormitory in enumerate(dorm):
        if dormitory == -1:
            return(idx)
    
    # the function should never be called if there are no not-assigned nodes left
    return

def kidOrganizer(G, dorm, s):
    '''
    Organize the kids that belongs to the same connected components as the node s
    and returns the number of assigned kids
    
    input: adjacency list of graph,
           dormitory list,
           starting node
    '''
    
    Q = []  # this list will be used as a stack
    
    dorm[s] = 0    # we assign the first kid to dorm 0
    
    # push s in the stack
    Q.append(s)
    
    processed_nodes = 0
    while len(Q)>0:
        
        v = Q.pop()
        processed_nodes += 1
        
        for u in G[v]:
            
            if dorm[u]==-1:
                dorm[u] = (dorm[v] + 1)%2  # changes the dorm from 0 to 1, or from 1 to 0
                Q.append(u)   # push u in the stack
                
            elif dorm[u] == dorm[v]:
                return(0)
    
    return(processed_nodes)

The while cycle in the algorithm is repeated once for every connected components in the graph; denoting with $n_i$ and $k_i$ the number of nodes and edges of the $i^{th}$ connected component, then the $i^{th}$ iteration is completed in $O(\sum_{j=0}^{i-1}n_j + k_i)$ steps (it takes at most a number of steps equal to the total visited nodes to execute ```getNextNode()```, and at most $O(k_j)$ to execute ```kidOrganizer()```). So the complexity of the algorithm is at most $O(K(n+k))$ where $K$ is the number of connected components of $G$.

In [24]:
G=[[1,3,5],[0,2],[1],[0,5],[],[0,3]]   # there is a 3-cycle (1,3,5) here, so there is no solution
dorm = general_kidOrganizer(G)
for idx, dorm in enumerate(dorm):
    print(f'Kid {idx} goes in dorm {dorm}')

There is no solution for the given graph, the kids are doomed to fight until just one (or maybe more) remains.

Welcome to the thunderdome.



In [25]:
G=[[1,5],[0,2],[1],[5],[],[0,3]]      # this graph is composed by two disjoint trees, so there is a solution
dorm = general_kidOrganizer(G)
for idx, dorm in enumerate(dorm):
    print(f'Kid {idx} goes in dorm {dorm}')

Kid 0 goes in dorm 0
Kid 1 goes in dorm 1
Kid 2 goes in dorm 0
Kid 3 goes in dorm 0
Kid 4 goes in dorm 0
Kid 5 goes in dorm 1
